Based on `04-.../23-latest_ak_slg_prepl_sweep/prep0.5/` introduce node entry/exit

In [1]:
using Plots, Distributions, StatsBase, IJuliaBell, IJuliaBell
# import Pkg; Pkg.add("CSV")
using CSV

In [2]:
import Random
Random.seed!(6820)

Random.TaskLocalRNG()

In [3]:
include("./src/rewiring_model.jl")

import .temp_vat

In [4]:
include("./src/model_analytics_utils.jl")
import .temp_vat_analytics

In [5]:
include("./src/model_init_utils.jl")
import .temp_vat_initialize_run

In [6]:
# load nodelist edgelist

In [7]:
# initialize NW
path_el = "./data/config_1701_edgelist_with_zeros.csv" # GBD: "/home/reisch/temporalVATnetwork/model_building/data/config_1701_edgelist_with_zeros.csv" # index_col=0)
path_nl = "./data/config_1701_nodelist_with_zeros.csv" # GBD: "/home/reisch/temporalVATnetwork/model_building/data/config_1701_nodelist_with_zeros.csv" #, index_col=0)
nodelist, edgelist, nace_consecutive_original_counts, nace_to_consecutive, consecutive_to_nace, Nnace, N0, L0 = temp_vat_initialize_run.load_init_pnw(path_nl, path_el);

# load att_mat
path_att_mat = "./data/att_mat_2_new-long.csv" # GBD: '/home/reisch/temporalVATnetwork/model_building/data/att_mat_2_new-long.csv')
const am2 = temp_vat_initialize_run.load_att_mat(path_att_mat, Nnace, nace_to_consecutive);
const am2_T = temp_vat_initialize_run.load_att_mat_T(path_att_mat, Nnace, nace_to_consecutive);

# define FDA
function fda(k)
    if k==0
        return 0.00001 # 43.9849
    else
        return k^1.079 # for kout
    end
end

# # define NSP funcition:
# path_nsp_persec = "../../../data/consistent_nw/slg_parameters_2017_consistent_sector_dummy_ktot.csv"

# nsp_params = Dict{Int,Tuple{Float64, Float64, Float64}}()
# for row in CSV.File(path_nsp_persec)
# #     println(row)
#     if row[:Column1] in keys(nace_to_consecutive)
#         n = nace_to_consecutive[row[:Column1]]
#         push!(nsp_params, (n => (row[:a0], row[:intercept], row[:alpha])))
#     end
# end

# const nsp_params_const = nsp_params

nsp_expected(k, nace) = 0.0117 * k^1.0495#50
# nsp_expected(k, nace) = 0.00708 * k^1.117

# function nsp_expected(k, nace)
#     (a0, intercept, alpha) = nsp_params_const[nace]
#     return intercept* k^1.0 #alpha
# end

nsp(k, nace, N) = rand(Distributions.Binomial(N, nsp_expected(k, nace)/N),1)


### new nodes function:
# helper fct for power-law:
function rndm(a, b, g, size=1)
#     """Power-law gen for pdf(x)\propto x^{g-1} for a<=x<=b"""
    r = rand(size)
    ag, bg = a^g, b^g
    return Int.(floor.((ag .+ (bg .- ag).*r).^(1. / g)))
end
# xx = rndm(1., 1e8, -3.5, 750)

rndm (generic function with 2 methods)

In [8]:
run_name = "run_2017_init_NW"
temp_vat.save_run_ESRI_compatible(run_name, nodelist, edgelist, consecutive_to_nace)


"edgelist_run_2017_init_NW.csv"

In [9]:
# new nodes fct:
N_entry = 349 # TRUE: 346
path_entry_table = "./data/kin_kout_table_entry_maxkout3.csv"

# entry_ktot_max = 3
kvec = Vector{Tuple{Int,Int}}()
pvec = Vector{Float64}()
for row in CSV.File(path_entry_table)
    kin=row[:kin]
    kout=row[:kout]
    prob = row[:joint_prob_reduced]
    push!(kvec, (kin,kout))
    push!(pvec,prob)
end

function new_nodes_fct(n_entry, nodelist, nace_prob_dict, kvec, pvec)

    kin_kout_combination = sample(kvec, Weights(pvec), n_entry, replace=true)
    
    naces = collect(keys(nace_prob_dict))
    nace_prob = getindex.(Ref(nace_prob_dict), naces)
    nace_vec = StatsBase.sample(naces, StatsBase.Weights(nace_prob), n_entry)

    new_nodes = [[nace, dangling_k[1], dangling_k[2]] for (nace, dangling_k) in zip(nace_vec, kin_kout_combination)]

    return new_nodes
end

# nnf depending on nodelist:
nnf(nodelist) = new_nodes_fct(N_entry, nodelist, nace_consecutive_original_counts, kvec, pvec)

nnf (generic function with 1 method)

In [10]:
const kin0, kout0, ktot0 = temp_vat.calc_degree_hist(nodelist)

(Dict{Any, Any}(5 => 266, 35 => 4, 55 => 1, 110 => 1, 60 => 2, 30 => 3, 32 => 4, 6 => 173, 45 => 2, 233 => 1…), Dict{Any, Any}(5 => 210, 56 => 1, 35 => 2, 220 => 1, 32 => 3, 6 => 121, 45 => 3, 64 => 1, 4 => 346, 13 => 21…), Dict{Any, Any}(5 => 439, 56 => 2, 35 => 3, 60 => 1, 30 => 10, 32 => 4, 6 => 304, 67 => 1, 45 => 3, 117 => 1…))

In [11]:
path_entry_table = "./data/kin_kout_table_entry_maxkout3.csv"

# entry_ktot_max = 3
kvec = Vector{Tuple{Int,Int}}()
pvec = Vector{Float64}()
for row in CSV.File(path_entry_table)
    kin=row[:kin]
    kout=row[:kout]
    prob = row[:joint_prob_reduced]
    push!(kvec, (kin,kout))
    push!(pvec,prob)
end

In [12]:
# set parameters:
# Nentry=602
p_node_exit = 0.00492 # 0.015 # 0.0417
pterm=0.02138#-p_node_exit
preplace=0.00


0.0

In [13]:
L0/N0

1.5097580430736506

In [ ]:
# run sim:
N_t, L_t = temp_vat.run_sim(nodelist, edgelist, pterm, preplace, p_node_exit, nsp, Nnace, am2, am2_T, fda, nnf, 500);

 --- Timestep 1 --- 
Added 349 new nodes.
Removed 251 nodes.
Removed 131 nodes.
No egligble targets.
No egligble targets.
No egligble targets.
ntot_dangling before: 883, and after: 647
Removed 3 nodes.
 --- Timestep 2 --- 
Added 349 new nodes.
Removed 232 nodes.
Removed 140 nodes.
No egligble targets.
No egligble targets.
No egligble targets.
No egligble targets.
No egligble targets.
ntot_dangling before: 862, and after: 608
Removed 5 nodes.
 --- Timestep 3 --- 
Added 349 new nodes.
Removed 224 nodes.
Removed 118 nodes.
No egligble targets.
No egligble targets.
No egligble targets.
ntot_dangling before: 851, and after: 609
Removed 3 nodes.
 --- Timestep 4 --- 
Added 349 new nodes.
Removed 251 nodes.
Removed 162 nodes.
No egligble targets.
ntot_dangling before: 878, and after: 647
Removed 1 nodes.
 --- Timestep 5 --- 
Added 349 new nodes.
Removed 209 nodes.
Removed 130 nodes.
No egligble targets.
No egligble targets.
No egligble targets.
ntot_dangling before: 854, and after: 612
Removed

In [ ]:
L_t[end]/N_t[end]

In [ ]:
plot(N_t)
plot!([0,500], [N0, N0])

In [ ]:
plot(L_t)
plot!([0,500], [L0, L0])

In [ ]:
kin, kout, ktot = temp_vat.calc_degree_hist(nodelist)

In [ ]:
temp_vat_analytics.my_pdf(kin0, kout0, "kin original NW", "kout original NW", "in- or outdegree")

In [ ]:
temp_vat_analytics.my_pdf(kin, kout, "kin synthetic", "kout synthetic", "in- or outdegree")

In [ ]:
temp_vat_analytics.my_ccdf(kin0, kout0, "kin original NW", "kout original NW", "in- or outdegree")

In [ ]:
temp_vat_analytics.my_ccdf(kin, kout, "kin synthetic", "kout synthetic", "in- or outdegree")

In [ ]:
temp_vat_analytics.my_pdf(kin0,kin, "original NW", "synthetic NW", "indegree")

In [ ]:
temp_vat_analytics.my_ccdf(kin0, kin, "original NW", "synthetic NW", "indegree, kin")

In [ ]:
temp_vat_analytics.my_pdf(kout0, kout, "original NW", "synthetic NW", "outdegree, kout")

In [ ]:
temp_vat_analytics.my_ccdf(kout0, kout, "original NW", "synthetic NW", "outdegree, kout")

In [ ]:
temp_vat_analytics.my_ccdf(ktot0, ktot, "original NW", "synthetic NW", "outdegree, ktot")
# @bell sleep(0.1)

In [ ]:
results = Dict()

# look at variation:
for i in 1:10
    N_t_, L_t_ = temp_vat.run_sim(nodelist, edgelist, pterm, preplace, p_node_exit, nsp, Nnace, am2, am2_T, fda, nnf, 200);
    kin, kout, ktot = temp_vat.calc_degree_hist(nodelist)
    
    results[i] = [kin, kout, ktot, N_t_, L_t_]
    
    run_name = "run_2017_$i"
    temp_vat.save_run_ESRI_compatible(run_name, nodelist, edgelist, consecutive_to_nace)
end

In [ ]:
results

In [ ]:
function my_ccdf(dic1, dic2, label1="vec 1", label2="vec 2", xlab="x lab")    
    x1,ccdf1 = calc_ccdf(dic1)
    pushfirst!(ccdf1, 1)
    plot(x1 .+1, ccdf1[1:end-1], xaxis=:log10, yaxis=:log10, label=label1, xlabel=xlab, ylabel="CCDF, p(X ≥ x)")
    
    x2,ccdf2 = calc_ccdf(dic2)
    pushfirst!(ccdf2,1)
    plot!(x2 .+1, ccdf2[1:end-1], xaxis=:log10, yaxis=:log10, label=label2)
end


In [ ]:
x, ccdf = temp_vat_analytics.calc_ccdf(ktot0)
pushfirst!(ccdf, 1)

p = plot(x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10, label="original NW", xlabel="degree, ktot", ylabel="CCDF, p(X ≥ x)", lw=5)

for r in values(results)
    x, ccdf = temp_vat_analytics.calc_ccdf(r[3])
    pushfirst!(ccdf, 1)
    plot!(p, x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10)
end
display(p)

In [ ]:
x, ccdf = temp_vat_analytics.calc_ccdf(kin0)
pushfirst!(ccdf, 1)

p = plot(x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10, label="original NW", xlabel="indegree, kin", ylabel="CCDF, p(X ≥ x)", lw=5)

for r in values(results)
    x, ccdf = temp_vat_analytics.calc_ccdf(r[1])
    pushfirst!(ccdf, 1)
    plot!(p, x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10)
end
display(p)

In [ ]:
x, ccdf = temp_vat_analytics.calc_ccdf(ktot0)
pushfirst!(ccdf, 1)

p = plot(x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10, label="original NW", xlabel="outdegree, kout", ylabel="CCDF, p(X ≥ x)", lw=5)

for r in values(results)
    x, ccdf = temp_vat_analytics.calc_ccdf(r[3])
    pushfirst!(ccdf, 1)
    plot!(p, x .+1, ccdf[1:end-1], xaxis=:log10, yaxis=:log10)
end
display(p)

In [ ]:
length(L_t)

In [ ]:
L_t_long = deepcopy(L_t) #Vector{Float64}()

for i in 1:10
    Lt_temp = results[i][5];
    append!(L_t_long, Lt_temp);
end


plot(L_t_long, xlab="timestep t", ylab="number of links L(t)")
vline!([500+200*i for i in 1:10])
plot!([0,2500], [L0, L0])

In [ ]:
plot(L_t_long./L0)
vline!([500+200*i for i in 1:10])
plot!([0,2500], [1,1])

In [ ]:
plot(L_t_long./L0)
vline!([500+200*i for i in 1:10])
plot!([0,2500], [1,1])
# plot!([0,2000], [L0, L0])

In [ ]:
N_t_long = deepcopy(N_t) # Vector{Float64}()

for i in 1:10
    Nt_temp = results[i][4];
    append!(N_t_long, Nt_temp);
end


plot(N_t_long, xlab="timestep t", ylab="number of nodes N(t)")
vline!([500+200*i for i in 1:10])
plot!([0,2500], [N0, N0])

In [ ]:
plot(L_t_long./N_t_long)
vline!([500+200*i for i in 1:10])
plot!([0,2500], [L0/N0, L0/N0])

In [ ]:
@bell sleep(0.1)

In [ ]:
L_t_long

In [ ]:
using DataFrames
function save_run_timeseries(run_name, N_t_long, L_t_long)
    df_NLt = DataFrame([(ts=i, Lt=L_t_long[i], Nt=N_t_long[i]) for i in 1:length(L_t_long)]);
    CSV.write("timeseries_N_L-$run_name.csv", df_NLt)
end

In [ ]:
save_run_timeseries("run_2017", N_t_long, L_t_long)

In [ ]:
N0

In [ ]:
L0